---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [79]:
import pandas as pd
import numpy as np
train_set = pd.read_csv('readonly/train.csv', encoding='ISO-8859-1')
prediction_set =  pd.read_csv('readonly/test.csv', encoding="ISO-8859-1")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [80]:
train_set.head(3)
# len(test_set)

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0
2,22062,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","SANDERS, DERRON",1449.0,LONGFELLOW,NaN,23658.0,P.O. BOX,DETROIT,...,0.0,0.0,0.0,0.0,NaN,NO PAYMENT APPLIED,NaN,NaN,not responsible by disposition,NaN


In [81]:
prediction_set.head()

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,violation_description,disposition,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status
0,284932,Department of Public Works,"Granberry, Aisha B","FLUELLEN, JOHN A",10041.0,ROSEBERRY,NaN,141,ROSEBERRY,DETROIT,...,Failure to secure City or Private solid waste ...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
1,285362,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,1000.0,20.0,10.0,100.0,0.0,0.0,1130.0,NaN
2,285361,Department of Public Works,"Lusk, Gertrina","WHIGHAM, THELMA",18520.0,EVERGREEN,NaN,19136,GLASTONBURY,DETROIT,...,Improper placement of Courville container betw...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN
3,285338,Department of Public Works,"Talbert, Reginald","HARABEDIEN, POPKIN",1835.0,CENTRAL,NaN,2246,NELSON,WOODHAVEN,...,Allowing bulk solid waste to lie or accumulate...,Responsible by Default,200.0,20.0,10.0,20.0,0.0,0.0,250.0,NaN
4,285346,Department of Public Works,"Talbert, Reginald","CORBELL, STANLEY",1700.0,CENTRAL,NaN,3435,MUNGER,LIVONIA,...,Violation of time limit for approved container...,Responsible by Default,100.0,20.0,10.0,10.0,0.0,0.0,140.0,NaN


In [82]:
len(prediction_set.columns)

27

In [83]:
len(train_set.columns)

34

In [87]:
# Extracting the column thaat are not common to both training and test set
x = []
for column in train_set.columns:
    if column not in prediction_set.columns:
        x.append(column)

extra_feature_train_set = x[:-1]
extra_feature_train_set

[]

In [88]:
# Removing extra feature from train set that are not in test set
train_set.drop(extra_feature_train_set, inplace= True, axis=1)
train_set.dropna(axis ='index', subset=['compliance'], inplace=True)

In [89]:
# More filtering wrt address and Latitude, Longitude
# On training data
address = pd.read_csv('readonly/addresses.csv')
lat_lon = pd.read_csv('readonly/latlons.csv')

In [90]:
address.head(3)

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"


In [91]:
lat_lon['lat'] = lat_lon['lat'].astype(float)
lat_lon['lon'] = lat_lon['lon'].astype(float)
lat_lon.fillna(method ='pad', axis=0, inplace= True)
train_set = address.merge(train_set, how='inner', on ='ticket_id')
train_set = train_set.merge(lat_lon, how='inner', on ='address')

# On test data
prediction_set = address.merge(prediction_set, how='inner', on ='ticket_id')
prediction_set = prediction_set.merge(lat_lon, how='inner', on ='address')

In [92]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 0 to 159879
Data columns (total 31 columns):
ticket_id                     159880 non-null int64
address                       159880 non-null object
agency_name                   159880 non-null object
inspector_name                159880 non-null object
violator_name                 159854 non-null object
violation_street_number       159880 non-null float64
violation_street_name         159880 non-null object
violation_zip_code            0 non-null float64
mailing_address_str_number    157322 non-null float64
mailing_address_str_name      159877 non-null object
city                          159880 non-null object
state                         159796 non-null object
zip_code                      159879 non-null object
non_us_str_code               3 non-null object
country                       159880 non-null object
ticket_issued_date            159880 non-null object
hearing_date                  159653 non-null obj

In [93]:
lat_lon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121769 entries, 0 to 121768
Data columns (total 3 columns):
address    121769 non-null object
lat        121769 non-null float64
lon        121769 non-null float64
dtypes: float64(2), object(1)
memory usage: 2.8+ MB


In [94]:
# Extract no of different features per column
distinct_labels = dict()
for column in train_set.columns:
    distinct_labels.update( {column : len(train_set[column].value_counts())} )
distinct_labels  

{'address': 71901,
 'admin_fee': 1,
 'agency_name': 5,
 'city': 4093,
 'clean_up_cost': 1,
 'compliance': 2,
 'country': 5,
 'discount_amount': 13,
 'disposition': 4,
 'fine_amount': 40,
 'grafitti_status': 0,
 'hearing_date': 5970,
 'inspector_name': 159,
 'judgment_amount': 57,
 'lat': 61559,
 'late_fee': 37,
 'lon': 66839,
 'mailing_address_str_name': 28440,
 'mailing_address_str_number': 14090,
 'non_us_str_code': 2,
 'state': 59,
 'state_fee': 1,
 'ticket_id': 159880,
 'ticket_issued_date': 68097,
 'violation_code': 189,
 'violation_description': 207,
 'violation_street_name': 1716,
 'violation_street_number': 18096,
 'violation_zip_code': 0,
 'violator_name': 84656,
 'zip_code': 4622}

In [95]:
# Extracting some relevent column using knowledge
columns_dummy = ['violation_street_number',
           'fine_amount', 'admin_fee', 'state_fee', 'late_fee',
           'discount_amount', 'clean_up_cost', 'judgment_amount',
           'lat', 'lon', 'compliance']
data_set_dummy = train_set[columns_dummy]
data_set_dummy.head()

,violation_street_number,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,lat,lon,compliance
0,2900.0,250.0,20.0,10.0,25.0,0.0,0.0,305.0,42.390729,-83.124268,0.0
1,2900.0,500.0,20.0,10.0,50.0,0.0,0.0,580.0,42.390729,-83.124268,0.0
2,2900.0,250.0,20.0,10.0,25.0,0.0,0.0,305.0,42.390729,-83.124268,0.0
3,2900.0,100.0,20.0,10.0,10.0,0.0,0.0,140.0,42.390729,-83.124268,0.0
4,2900.0,300.0,20.0,10.0,30.0,0.0,0.0,360.0,42.390729,-83.124268,0.0


In [96]:
# Splitting X_train & y_train
from sklearn.model_selection import train_test_split
X_dummy = data_set_dummy.iloc[:, :-1]
y_dummy = data_set_dummy.iloc[:, -1]
X_train_dummy, X_test_dummy, y_train_dummy, y_test_dummy = train_test_split(X_dummy, y_dummy, test_size=.2)

In [97]:
# Finding the feature importance
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier()
classifier.fit(X_train_dummy, y_train_dummy)
importance = classifier.feature_importances_
feature_importance  = list(zip(X_train_dummy.columns, importance))
feature_importance
# Sorting the columns as per feature importance
feature, importance = list(), list()
for i in range(len(feature_importance)):
    feature.append(feature_importance[i][0])
    importance.append(feature_importance[i][1])
    
feature_importance = pd.Series(data = importance, index = feature).sort_values(ascending= False)
feature_importance

lon                        0.273095
lat                        0.245621
violation_street_number    0.233876
late_fee                   0.157154
discount_amount            0.044696
judgment_amount            0.025358
fine_amount                0.020200
clean_up_cost              0.000000
state_fee                  0.000000
admin_fee                  0.000000
dtype: float64

In [98]:
# Extracted final training set for trainign data
feature_importance_column = ['ticket_id','lon', 'lat', 'violation_street_number', 'late_fee', 
                             'discount_amount','judgment_amount', 'compliance']    
final_data_set = train_set[feature_importance_column]
final_data_set.set_index('ticket_id', inplace= True)

In [99]:
final_data_set.head(3)

,lon,lat,violation_street_number,late_fee,discount_amount,judgment_amount,compliance
ticket_id,,,,,,,
22056,-83.124268,42.390729,2900.0,25.0,0.0,305.0,0.0
77242,-83.124268,42.390729,2900.0,50.0,0.0,580.0,0.0
77243,-83.124268,42.390729,2900.0,25.0,0.0,305.0,0.0


In [100]:
final_data_set.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 22056 to 285125
Data columns (total 7 columns):
lon                        159880 non-null float64
lat                        159880 non-null float64
violation_street_number    159880 non-null float64
late_fee                   159880 non-null float64
discount_amount            159880 non-null float64
judgment_amount            159880 non-null float64
compliance                 159880 non-null float64
dtypes: float64(7)
memory usage: 9.8 MB


In [101]:
# Extracting same feature from Prediction set

X_prediction = prediction_set[feature_importance_column[:-1]]
X_prediction.set_index('ticket_id', inplace= True)


In [102]:
X_prediction.head(3)

,lon,lat,violation_street_number,late_fee,discount_amount,judgment_amount
ticket_id,,,,,,
284932,-82.986642,42.407581,10041.0,20.0,0.0,250.0
285362,-83.238259,42.426239,18520.0,100.0,0.0,1130.0
285361,-83.238259,42.426239,18520.0,10.0,0.0,140.0


In [103]:
X = final_data_set.iloc[:, :-1]
y = final_data_set.iloc[:, -1]

In [104]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [105]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Choosing best model for learning and Do hyperparameter tuning

In [106]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

model_params = {
                 'random_forest':{ 'model': RandomForestClassifier(),
                                   'params': {'n_estimators': [10,50,100]}} ,
                 'decision_tree': { 'model': DecisionTreeClassifier(),
                                    'params': {'min_samples_split': [3,5,7]}}
                }
scores = list()

for model_name, parameter in model_params.items():
    clf = GridSearchCV(parameter['model'], param_grid= parameter['params'], cv=5, scoring='roc_auc' )
    clf.fit(X_train, y_train)
    scores.append({'model': model_name, 'best_score': clf.best_score_, 'best_params': clf.best_params_})
scores

[{'best_params': {'n_estimators': 100},
  'best_score': 0.78611712917376875,
  'model': 'random_forest'},
 {'best_params': {'min_samples_split': 7},
  'best_score': 0.6678000749384756,
  'model': 'decision_tree'}]

# Trainig the model with best obtained classifier and its parameter

In [107]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
clf1 = RandomForestClassifier(n_estimators =100)
clf1.fit(X_train, y_train)
 
# y_pred = clf1.predict(X_test)
# print('roc_auc = {:.2f}'.format(roc_auc_score(y_test, y_pred)))
# print('accuracy = {:.2f}'.format(accuracy_score(y_test, y_pred)))

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [108]:
auc_score = cross_val_score(clf1, X_train, y_train, cv=5, scoring='roc_auc')
train_set_accuracy = cross_val_score(clf1, X_train, y_train, cv=5, scoring='accuracy')
test_set_accuracy = cross_val_score(clf1, X_test, y_test, cv=5, scoring='accuracy')

In [112]:
print(auc_score)
print(train_set_accuracy)
print(test_set_accuracy)

[ 0.78783316  0.78813137  0.7924187   0.78912779  0.77320317]
[ 0.92838435  0.93076893  0.92916618  0.92975255  0.92834246]
[ 0.92995622  0.93058161  0.92744332  0.92603597  0.92774476]


In [115]:
print('Best score(AUC): {:.3f}'.format(auc_score.mean()))
print('Train set accuracy: {:.3f}'.format(train_set_accuracy.mean()))
print('Test set accuracy: {:.3f}'.format(test_set_accuracy.mean()))

Best score(AUC): 0.786
Train set accuracy: 0.929
Test set accuracy: 0.928


In [116]:
# Y Predicted probabilistic
y_prediction_proba = clf1.predict_proba(X_prediction)
y_proba = list()
for i in range(len(y_prediction_proba)):
    y_proba.append(y_prediction_proba[i][1])

In [117]:
pd.DataFrame(y_proba, index= X_prediction.index)

,0
ticket_id,
284932,0.070000
285362,0.000000
285361,0.000000
285338,0.000000
285346,0.010000
285345,0.010000
285347,0.070000
285342,0.120000
285530,0.016667


In [45]:
import pandas as pd
import numpy as np

def blight_model():
    
    train_set = pd.read_csv('readonly/train.csv', encoding='ISO-8859-1')
    prediction_set =  pd.read_csv('readonly/test.csv', encoding="ISO-8859-1")
    
    # Extracting the column thaat are not common to both training and test set
    x = []
    for column in train_set.columns:
        if column not in prediction_set.columns:
            x.append(column)

    extra_feature_train_set = x[:-1]
    extra_feature_train_set
    # Removing extra feature from train set that are not in test set
    train_set.drop(extra_feature_train_set, inplace= True, axis=1)
    train_set.dropna(axis ='index', subset=['compliance'], inplace=True)
    # More filtering wrt address and Latitude, Longitude
    # On training data
    address = pd.read_csv('readonly/addresses.csv')
    lat_lon = pd.read_csv('readonly/latlons.csv')
    lat_lon['lat'] = lat_lon['lat'].astype(float)
    lat_lon['lon'] = lat_lon['lon'].astype(float)
    lat_lon.fillna(method ='pad', axis=0, inplace= True)
    train_set = address.merge(train_set, how='inner', on ='ticket_id')
    train_set = train_set.merge(lat_lon, how='inner', on ='address')

    # On test data
    prediction_set = address.merge(prediction_set, how='inner', on ='ticket_id')
    prediction_set = prediction_set.merge(lat_lon, how='inner', on ='address')
    
    # Extracting some relevent column using knowledge
    columns_dummy = ['violation_street_number',
               'fine_amount', 'admin_fee', 'state_fee', 'late_fee',
               'discount_amount', 'clean_up_cost', 'judgment_amount',
               'lat', 'lon', 'compliance']
    data_set_dummy = train_set[columns_dummy]
    
    # Splitting X_train & y_train
    from sklearn.model_selection import train_test_split
    X_dummy = data_set_dummy.iloc[:, :-1]
    y_dummy = data_set_dummy.iloc[:, -1]
    X_train_dummy, X_test_dummy, y_train_dummy, y_test_dummy = train_test_split(X_dummy, y_dummy, test_size=.2)
    
    # Finding the feature importance
    from sklearn.tree import DecisionTreeClassifier
    classifier = DecisionTreeClassifier()
    classifier.fit(X_train_dummy, y_train_dummy)
    importance = classifier.feature_importances_
    feature_importance  = list(zip(X_train_dummy.columns, importance))
    feature_importance
    # Sorting the columns as per feature importance
    feature, importance = list(), list()
    for i in range(len(feature_importance)):
        feature.append(feature_importance[i][0])
        importance.append(feature_importance[i][1])

    feature_importance = pd.Series(data = importance, index = feature).sort_values(ascending= False)
    feature_importance
    
    # Extracted final training set for trainign data as per feature importance score
    feature_importance_column = ['ticket_id','lon', 'lat', 'violation_street_number', 'late_fee', 
                                 'discount_amount','judgment_amount', 'compliance']    
    final_data_set = train_set[feature_importance_column]
    final_data_set.set_index('ticket_id', inplace= True)
    
    # Extracting same feature from Prediction set
    X_prediction = prediction_set[feature_importance_column[:-1]]
    X_prediction.set_index('ticket_id', inplace= True)
    # Final estimation of model performence
    # Splitting X and Y
    X = final_data_set.iloc[:, :-1]
    y = final_data_set.iloc[:, -1]
    # Splitting train and test set
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)
    
    # Tranign on best fitted model rerived by Hyper parameter tuning (See rough code outside of function )
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import cross_val_score
    clf1 = RandomForestClassifier(n_estimators =100)
    clf1.fit(X_train, y_train)
    # ROC_AUC score of the model selected
    cross_val_score = cross_val_score(clf1, X_train, y_train, cv=5, scoring='roc_auc')
    'Best score(AUC): {:.3f}'.format(cross_val_score.mean()) 
    
    # Y Predicted probabilistic
    y_prediction_proba = clf1.predict_proba(X_prediction)
    y_proba = list()
    for i in range(len(y_prediction_proba)):
        y_proba.append(y_prediction_proba[i][1])
        
    return pd.DataFrame(y_proba, index= X_prediction.index)


In [46]:
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,0
ticket_id,
284932,0.010000
285362,0.000000
285361,0.000000
285338,0.050000
285346,0.010000
285345,0.030000
285347,0.020000
285342,0.050000
285530,0.005000
